## Estudo acoes

### Bibilioteca

In [126]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import os
import pickle
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import pickle
from datetime import datetime
import requests
from sklearn.model_selection import train_test_split


### Busca e tratamento de dados

In [176]:
# URL base da API
base_url = "https://statusinvest.com.br/category/advancedsearchresultpaginated"

# Parâmetros da busca
search_params = {
    "search": '{"Sector":"","SubSector":"","Segment":"","my_range":"-20;100","forecast":{"upsidedownside":{"Item1":null,"Item2":null},"estimatesnumber":{"Item1":null,"Item2":null},"revisedup":true,"reviseddown":true,"consensus":[]},"dy":{"Item1":null,"Item2":null},"p_l":{"Item1":null,"Item2":null},"peg_ratio":{"Item1":null,"Item2":null},"p_vp":{"Item1":null,"Item2":null},"p_ativo":{"Item1":null,"Item2":null},"margembruta":{"Item1":null,"Item2":null},"margemebit":{"Item1":null,"Item2":null},"margemliquida":{"Item1":null,"Item2":null},"p_ebit":{"Item1":null,"Item2":null},"ev_ebit":{"Item1":null,"Item2":null},"dividaliquidaebit":{"Item1":null,"Item2":null},"dividaliquidapatrimonioliquido":{"Item1":null,"Item2":null},"p_sr":{"Item1":null,"Item2":null},"p_capitalgiro":{"Item1":null,"Item2":null},"p_ativocirculante":{"Item1":null,"Item2":null},"roe":{"Item1":null,"Item2":null},"roic":{"Item1":null,"Item2":null},"roa":{"Item1":null,"Item2":null},"liquidezcorrente":{"Item1":null,"Item2":null},"pl_ativo":{"Item1":null,"Item2":null},"passivo_ativo":{"Item1":null,"Item2":null},"giroativos":{"Item1":null,"Item2":null},"receitas_cagr5":{"Item1":null,"Item2":null},"lucros_cagr5":{"Item1":null,"Item2":null},"liquidezmediadiaria":{"Item1":null,"Item2":null},"vpa":{"Item1":null,"Item2":null},"lpa":{"Item1":null,"Item2":null},"valormercado":{"Item1":null,"Item2":null}}',
    "orderColumn": "",
    "isAsc": "",
    "page": 0,  # Começamos da página 0
    "take": 15,  # Número de itens por página
    "CategoryType": 1
}

# Cabeçalhos da solicitação
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Lista para armazenar os DataFrames de cada página
all_dfs = []

# Loop para iterar sobre todas as páginas
while True:
    response = requests.get(base_url, params=search_params, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        df = pd.json_normalize(data['list'])
        all_dfs.append(df)
        
        # Incrementando o número da página
        search_params["page"] += 1
    else:
        print(f"Erro ao acessar a URL: {response.status_code}")
        break
    
    # Verificando se há mais páginas
    if not data['list']:
        break

# Concatenando todos os DataFrames em um único DataFrame
df = pd.concat(all_dfs, ignore_index=True)

# Exibindo os resultados
print(df.head())


   companyid                     companyname  ticker  price   p_l  p_vp   
0        205  FERROVIA CENTRO-ATLANTICA S.A.   VSPT4   0.00  0.00  0.00  \
1        226               VESTE S.A. ESTILO   VSTE3  14.10  6.76  1.56   
2        485          VULCABRAS/AZALEIA S.A.   VULC3  18.20  8.80  2.50   
3        216          GRUPO CASAS BAHIA S.A.  VVAR11  14.58 -0.78  0.31   
4        216          GRUPO CASAS BAHIA S.A.   VVAR4  95.68 -5.09  2.05   

   p_ebit  p_ativo  ev_ebit  margembruta  ...  valormercado  segmentid   
0    0.00     0.00    -1.03         7.78  ...  7.126108e+08         16  \
1    7.63     0.95     8.87        64.99  ...  1.607407e+09         22   
2    9.13     1.77     9.15        40.55  ...  4.968896e+09         32   
3   -8.30     0.04   -46.21        28.99  ...  7.045695e+08         20   
4  -54.48     0.28   -46.21        28.99  ...  7.045695e+08         20   

   sectorid  subsectorid                  subsectorname   
0       1.0          6.0                     

In [177]:
df.head(3)

,companyid,companyname,ticker,price,p_l,p_vp,p_ebit,p_ativo,ev_ebit,margembruta,...,valormercado,segmentid,sectorid,subsectorid,subsectorname,segmentname,sectorname,dy,lucros_cagr5,liquidezmediadiaria
0,205,FERROVIA CENTRO-ATLANTICA S.A.,VSPT4,0.0,0.00,0.00,0.00,0.00,-1.03,7.78,...,7.126108e+08,16,1.0,6.0,Transporte,Transporte Ferroviário,Bens Industriais,NaN,NaN,NaN
1,226,VESTE S.A. ESTILO,VSTE3,14.1,6.76,1.56,7.63,0.95,8.87,64.99,...,1.607407e+09,22,2.0,8.0,Comércio,Tecidos. Vestuário e Calçados,Consumo Cíclico,0.82,-10.77,47866.68
2,485,VULCABRAS/AZALEIA S.A.,VULC3,18.2,8.80,2.50,9.13,1.77,9.15,40.55,...,4.968896e+09,32,2.0,13.0,Tecidos. Vestuário e Calçados,Calçados,Consumo Cíclico,16.67,24.47,26418639.71


In [178]:
df.fillna(0, inplace=True)

In [179]:
df.isna().sum()

companyid                         0
companyname                       0
ticker                            0
price                             0
p_l                               0
p_vp                              0
p_ebit                            0
p_ativo                           0
ev_ebit                           0
margembruta                       0
margemebit                        0
margemliquida                     0
p_sr                              0
p_capitalgiro                     0
p_ativocirculante                 0
giroativos                        0
roe                               0
roa                               0
roic                              0
dividaliquidapatrimonioliquido    0
dividaliquidaebit                 0
pl_ativo                          0
passivo_ativo                     0
liquidezcorrente                  0
peg_ratio                         0
receitas_cagr5                    0
vpa                               0
lpa                         

### Definição alvo e previsores

In [180]:
alvo = df.price

In [181]:
previsores = df.drop(['price', 'ticker', 'companyname', 'companyid', 'segmentid', 'sectorid', 'subsectorid', 'subsectorname', 'segmentname', 'sectorname'], axis=1)

In [182]:
previsores.head(2)

,p_l,p_vp,p_ebit,p_ativo,ev_ebit,margembruta,margemebit,margemliquida,p_sr,p_capitalgiro,...,passivo_ativo,liquidezcorrente,peg_ratio,receitas_cagr5,vpa,lpa,valormercado,dy,lucros_cagr5,liquidezmediadiaria
0,0.00,0.00,0.00,0.00,-1.03,7.78,-44.27,-67.63,0.00,0.00,...,0.71,0.75,0.00,7.49,13.44,-15.66,7.126108e+08,0.00,0.00,0.00
1,6.76,1.56,7.63,0.95,8.87,64.99,19.07,21.51,1.45,9.03,...,0.39,1.48,-0.05,-3.13,9.02,2.09,1.607407e+09,0.82,-10.77,47866.68


In [183]:
alvo.head(2)

0     0.0
1    14.1
Name: price, dtype: float64

In [184]:
print(previsores.shape)
print(alvo.shape)

(620, 28)
(620,)


### Escalonamento

In [185]:
scaler = StandardScaler()
previsores_standard_scaling = scaler.fit_transform(previsores)

# Aplicando MinMaxScaler
min_max_scaler = MinMaxScaler()
previsores_min_scaling = min_max_scaler.fit_transform(previsores)

# Criando a pasta 'variaveis' se não existir
if not os.path.exists('variaveis'):
    os.makedirs('variaveis')

# Salvando os previsores escalonados
with open('variaveis/previsores_standard_scaling.pickle', 'wb') as f:
    pickle.dump(previsores_standard_scaling, f)

with open('variaveis/previsores_min_scaling.pickle', 'wb') as f:
    pickle.dump(previsores_min_scaling, f)

with open('variaveis/alvo.pickle', 'wb') as f:
    pickle.dump(alvo, f)

with open('variaveis/previsores.pickle', 'wb') as f:
    pickle.dump(previsores, f)

### Looping de algoritimos

In [186]:
previsor_escolhido = 'previsores_standard_scaling'
with open(f'./variaveis/alvo.pickle', 'rb') as file:
    alvo = pickle.load(file)
with open(f'./variaveis/{previsor_escolhido}.pickle', 'rb') as file:
    previsores = pickle.load(file)

In [187]:
inicio_treinamento = datetime.now()

# Dividindo o conjunto em treino e teste
X_train, X_test, y_train, y_test = train_test_split(previsores, alvo, test_size=0.3, random_state=42)

# Diretório para salvar os modelos treinados
algoritmos_dir = "./modelos"

# Inicializando dicionário de algoritmos para regressão
algoritmos = {
    "random_forest": RandomForestRegressor(),
    "decision_tree": DecisionTreeRegressor(),
    "linear_regression": LinearRegression(),
    "catboost": CatBoostRegressor(verbose=0),
    "mlp": MLPRegressor(),
    "adaboost": AdaBoostRegressor(),
    "gradient_boosting": GradientBoostingRegressor(),
}

resultados = {}

for nome, modelo in algoritmos.items():
    # Validação cruzada
    cv_scores = cross_val_score(modelo, X_train, y_train, cv=5, scoring='r2')
    cv_r2 = np.mean(cv_scores)
    cv_r2_std = np.std(cv_scores)

    # Treinamento e predição
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)

    # Métricas de performance
    r2 = r2_score(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)

    resultados[nome] = {
        "cv_r2": cv_r2,
        "cv_r2_std": cv_r2_std,
        "r2": r2,
        "rmse": rmse,
    }

    with open(f'{algoritmos_dir}/{nome}.pickle', 'wb') as file:
        pickle.dump(modelo, file)

fim_treinamento = datetime.now()

# Consolidando resultados
resultados_completos = {
    "resultados": resultados,
    "inicio": inicio_treinamento.strftime('%Y-%m-%d %H:%M:%S'),
    "fim": fim_treinamento.strftime('%Y-%m-%d %H:%M:%S')
}

# Salvando os resultados completos
resultado_completo_df = "resultado_completo_df.pickle"
with open(f'{algoritmos_dir}/{resultado_completo_df}', 'wb') as file:
    pickle.dump(resultados_completos, file)

/Users/rafaelmunareto/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/rafaelmunareto/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/rafaelmunareto/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/rafaelmunareto/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


## Resultado

In [188]:
with open('./modelos/resultado_completo_df.pickle', 'rb') as file:
    resultados_completos = pickle.load(file)

# Imprimir os resultados de R^2 para referência
for nome, resultado in resultados_completos['resultados'].items():
    print(f"{nome}: R^2 no teste = {resultado['r2']}")

random_forest: R^2 no teste = -0.6121300222682176
decision_tree: R^2 no teste = 0.3887293394244016
linear_regression: R^2 no teste = -1610.7757569734338
catboost: R^2 no teste = 0.6492428124468959
mlp: R^2 no teste = -0.030638985904401617
adaboost: R^2 no teste = -0.05278464921016357
gradient_boosting: R^2 no teste = 0.4301893006846884


In [189]:
# URL base da API
base_url = "https://statusinvest.com.br/category/advancedsearchresultpaginated"

# Parâmetros da busca
search_params = {
    "search": '{"Sector":"","SubSector":"","Segment":"","my_range":"-20;100","forecast":{"upsidedownside":{"Item1":null,"Item2":null},"estimatesnumber":{"Item1":null,"Item2":null},"revisedup":true,"reviseddown":true,"consensus":[]},"dy":{"Item1":null,"Item2":null},"p_l":{"Item1":null,"Item2":null},"peg_ratio":{"Item1":null,"Item2":null},"p_vp":{"Item1":null,"Item2":null},"p_ativo":{"Item1":null,"Item2":null},"margembruta":{"Item1":null,"Item2":null},"margemebit":{"Item1":null,"Item2":null},"margemliquida":{"Item1":null,"Item2":null},"p_ebit":{"Item1":null,"Item2":null},"ev_ebit":{"Item1":null,"Item2":null},"dividaliquidaebit":{"Item1":null,"Item2":null},"dividaliquidapatrimonioliquido":{"Item1":null,"Item2":null},"p_sr":{"Item1":null,"Item2":null},"p_capitalgiro":{"Item1":null,"Item2":null},"p_ativocirculante":{"Item1":null,"Item2":null},"roe":{"Item1":null,"Item2":null},"roic":{"Item1":null,"Item2":null},"roa":{"Item1":null,"Item2":null},"liquidezcorrente":{"Item1":null,"Item2":null},"pl_ativo":{"Item1":null,"Item2":null},"passivo_ativo":{"Item1":null,"Item2":null},"giroativos":{"Item1":null,"Item2":null},"receitas_cagr5":{"Item1":null,"Item2":null},"lucros_cagr5":{"Item1":null,"Item2":null},"liquidezmediadiaria":{"Item1":null,"Item2":null},"vpa":{"Item1":null,"Item2":null},"lpa":{"Item1":null,"Item2":null},"valormercado":{"Item1":null,"Item2":null}}',
    "orderColumn": "",
    "isAsc": "",
    "page": 0,  # Começamos da página 0
    "take": 15,  # Número de itens por página
    "CategoryType": 1
}

# Cabeçalhos da solicitação
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Lista para armazenar os DataFrames de cada página
all_dfs = []

# Loop para iterar sobre todas as páginas
while True:
    response = requests.get(base_url, params=search_params, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        df = pd.json_normalize(data['list'])
        all_dfs.append(df)
        
        # Incrementando o número da página
        search_params["page"] += 1
    else:
        print(f"Erro ao acessar a URL: {response.status_code}")
        break
    
    # Verificando se há mais páginas
    if not data['list']:
        break

# Concatenando todos os DataFrames em um único DataFrame
nova_base = pd.concat(all_dfs, ignore_index=True)

# Exibindo os resultados
print(nova_base.head())


   companyid                     companyname  ticker  price   p_l  p_vp   
0        205  FERROVIA CENTRO-ATLANTICA S.A.   VSPT4   0.00  0.00  0.00  \
1        226               VESTE S.A. ESTILO   VSTE3  14.10  6.76  1.56   
2        485          VULCABRAS/AZALEIA S.A.   VULC3  18.20  8.80  2.50   
3        216          GRUPO CASAS BAHIA S.A.  VVAR11  14.58 -0.78  0.31   
4        216          GRUPO CASAS BAHIA S.A.   VVAR4  95.68 -5.09  2.05   

   p_ebit  p_ativo  ev_ebit  margembruta  ...  valormercado  segmentid   
0    0.00     0.00    -1.03         7.78  ...  7.126108e+08         16  \
1    7.63     0.95     8.87        64.99  ...  1.607407e+09         22   
2    9.13     1.77     9.15        40.55  ...  4.968896e+09         32   
3   -8.30     0.04   -46.21        28.99  ...  7.045695e+08         20   
4  -54.48     0.28   -46.21        28.99  ...  7.045695e+08         20   

   sectorid  subsectorid                  subsectorname   
0       1.0          6.0                     

In [190]:
nova_base.fillna(0, inplace=True)
previsores_nova_base = nova_base.drop(['price', 'ticker', 'companyname', 'companyid', 'segmentid', 'sectorid', 'subsectorid', 'subsectorname', 'segmentname', 'sectorname'], axis=1)
for column in previsores_nova_base.columns:
    if column != 'ticker':
        previsores_nova_base[column] = previsores_nova_base[column].astype(str).apply(lambda x: x.replace('.', '').replace(',', '.') if ',' in x else x)
        previsores_nova_base[column] = previsores_nova_base[column].astype(float)


In [191]:
scaler = StandardScaler()
previsores_nova_scalonados = scaler.fit_transform(previsores_nova_base)

In [192]:
previsores_nova_base = pd.DataFrame(previsores_nova_base, columns=previsores_nova_base.columns)
previsores_nova_base['ticker'] = nova_base['ticker']
resultados_predicoes = previsores_nova_base[['ticker']].copy()
preco_real = nova_base['price']


In [193]:
for nome, _ in resultados_completos['resultados'].items():
    with open(f'{algoritmos_dir}/{nome}.pickle', 'rb') as file:
        modelo = pickle.load(file)
        resultados_predicoes[f'Preco_{nome}'] = modelo.predict(previsores_nova_scalonados)

In [194]:
resultados_predicoes['Preco_Real'] = preco_real
for nome in resultados_completos['resultados'].keys():
    resultados_predicoes[f'Dif_{nome}'] = abs(nova_base['price'] - resultados_predicoes['Preco_Real'])

In [195]:
import matplotlib.pyplot as plt
import seaborn as sns

# Identificar o modelo com o melhor R^2
melhor_modelo = max(resultados_completos['resultados'], key=lambda x: resultados_completos['resultados'][x]['r2'])
print(f"Melhor modelo: {melhor_modelo}")

# Criar um gráfico de barras verticais para o modelo
# Adicionando a coluna de diferenças para o melhor modelo
resultados_predicoes['Diferenca'] = abs(resultados_predicoes['Preco_Real'] - resultados_predicoes[f'Preco_{melhor_modelo}'])

# Ordenando pela maior diferença
resultados_predicoes.sort_values('Diferenca', ascending=False, inplace=True)

# Formatando valores em reais
resultados_predicoes['Preco_Real'] = resultados_predicoes['Preco_Real'].apply(lambda x: f"R$ {x:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.'))
resultados_predicoes[f'Preco_{melhor_modelo}'] = resultados_predicoes[f'Preco_{melhor_modelo}'].apply(lambda x: f"R$ {x:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.'))
resultados_predicoes['Diferenca'] = resultados_predicoes['Diferenca'].apply(lambda x: f"R$ {x:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.'))

# Criando a tabela com as colunas relevantes
tabela_diferencas = resultados_predicoes[['ticker', 'Preco_Real', f'Preco_{melhor_modelo}', 'Diferenca']]
tabela_diferencas.reset_index(drop=True, inplace=True)

# Exibindo a tabela
print(tabela_diferencas.to_string(index=False))


Melhor modelo: catboost
ticker   Preco_Real Preco_catboost Diferenca
 MSRO3    R$ 702,50      R$ 240,82 R$ 461,68
 MOSI3      R$ 8,24      R$ 112,89 R$ 104,65
 EEEL4    R$ 410,00      R$ 309,14 R$ 100,86
 AZUL4     R$ 13,32       R$ 86,84  R$ 73,52
 GUAR4    R$ 151,00       R$ 94,28  R$ 56,72
 BSEV3      R$ 9,00       R$ 58,90  R$ 49,90
 PRNR3     R$ 11,68       R$ 57,15  R$ 45,47
 PGMN3      R$ 2,80       R$ 46,21  R$ 43,41
 HAPV3      R$ 3,82       R$ 46,07  R$ 42,25
 CASN3      R$ 9,61       R$ 51,60  R$ 41,99
 SPRI3     R$ 21,72       R$ 61,61  R$ 39,89
 MNPR3     R$ 19,50       R$ 58,75  R$ 39,25
 GPAR3     R$ 42,77       R$ 78,94  R$ 36,17
 MNDL3     R$ 41,00       R$ 76,56  R$ 35,56
 CASN4      R$ 8,37       R$ 43,45  R$ 35,08
 SPRI6     R$ 11,00       R$ 45,51  R$ 34,51
 NINJ3      R$ 5,16       R$ 36,38  R$ 31,22
 SPRI5      R$ 8,52       R$ 39,73  R$ 31,21
 FNCN3     R$ 14,25       R$ 42,75  R$ 28,50
 ELET5     R$ 99,00       R$ 73,99  R$ 25,01
 ENMT3     R$ 80,00       R$ 55